In [24]:
import requests
import time
from random import random
from bs4 import BeautifulSoup
import pandas as pd

data = []

def dict_builder(products):

            print(f"Currently scraping {len(products)} products from page {page}.")

            for product in products:

                #creates dictionary of product info:
                d = {}
                d['product_id'] = (product['href'].split('-')[-1])[:-5]    
                d['title'] = product.find('div', {'class':'name'}).text
                d['price'] = product.find('div', {'class':'price-discount__price'}).text
                d['image_url'] = product.img['src']
                d['free_shipping'] = bool(product.find('div', {'class':'badge-top'}))
                d['reviewer_tot'] = (product.find('div', {'class':'review'}).text).strip('()')
                data.append(d)
                page += 1

def tiki_scraper(category_url, page_start=1):

    

    while True:
        page = page_start
        
        # generates random delay between 0.5 and 1.5 seconds:
        delay = random() + 0.5

        # defines target page url 
        current = category_url + str(page)

        # general url business and creating soup obj:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

        r = requests.get(current, headers=headers)
        soup = BeautifulSoup(r.text, "html.parser")

        # populates list of all products on target page:
        products = soup.find_all("a", {"class": "product-item"}) 

        #checks if there are actually products on the page (i.e. if it has come to final page):
        if len(products) != 0:
            dict_builder()
        
        else:
            for i in range(1, 21):
                print(f"Scraping failed, trying again for the {i}th time.")
                dict_builder()

        #delays next loop initiation
        time.sleep(delay)


In [35]:
tiki_scraper('https://tiki.vn/laptop-may-vi-tinh/c1846?src=c.1846.hamburger_menu_fly_out_banner&page=',205)


Currently scraping 48 products from page 205.


In [65]:
table = pd.DataFrame(data)
table.tail()

,product_id,free_shipping,title,price,image_url,reviewer_tot
331,p455445,True,Bộ Kích Sóng Wifi Repeater 300Mbps TP-Link TL-...,239.000 ₫,https://salt.tikicdn.com/ts/upload/f3/74/46/f4...,699
332,p416911,True,TP-Link TL-WN823N - USB Wifi chuẩn N tốc độ 3...,139.000 ₫,https://salt.tikicdn.com/cache/280x280/ts/prod...,178
333,p3237473,True,Chuột Có Dây DareU LM130 1000DPI Multi-LED - H...,110.000 ₫,https://salt.tikicdn.com/cache/280x280/ts/prod...,11
334,p4747467,True,Màn Hình Dell P2319H 23inch FullHD 8ms 60Hz IP...,3.690.000 ₫,https://salt.tikicdn.com/cache/280x280/ts/prod...,151
335,p41167737,True,Ram PC Kingston HyperX Fury Black 8GB (1x8GB) ...,739.000 ₫,https://salt.tikicdn.com/ts/upload/75/34/d2/4a...,111
